***Importing all packages/libraries and datasets***

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
from concurrent.futures import ThreadPoolExecutor
import random
from sklearn.preprocessing import OneHotEncoder
fd_14 = pd.read_csv("./Stocks/2014_Financial_Data.csv")
fd_15 = pd.read_csv("./Stocks/2015_Financial_Data.csv")
fd_16 = pd.read_csv("./Stocks/2016_Financial_Data.csv")
fd_17 = pd.read_csv("./Stocks/2017_Financial_Data.csv")
fd_18 = pd.read_csv("./Stocks/2018_Financial_Data.csv")

In [ ]:
# print(fd_14.head())
# print(fd_15.head())
# print(fd_16.head())
# print(fd_17.head())
# print(fd_18.head())
fd_14.info()
fd_14.describe()
# print(fd_14[["Unnamed: 0", "Asset Growth", "2015 PRICE VAR [%]", "Class"]])

In [ ]:
object_cols = list(filter(lambda x: isinstance(fd_14[x][0], str), fd_14.columns))
object_cols

In [ ]:
print(fd_14['Sector'].unique())

df_class = fd_14['Class'].value_counts()
print(df_class)
sns.barplot(np.arange(len(df_class)), df_class)
plt.show()

***Attempting to clean data***

In [ ]:
# Dropped those columns with more than 50% of their elements as NULL
fd_14 = fd_14.dropna(thresh=len(fd_14)/2, axis=1)
print(fd_14.head())
print(fd_14.shape)

In [ ]:

fd_14.isnull().sum()

In [ ]:
print(fd_14.columns)

In [ ]:
# Test to see how many rows will be left if all NULL valued rows were removed
fd_14 = fd_14.dropna()
fd_14.drop(['Unnamed: 0', 'Sector'], axis=1, inplace=True)
fd_14.to_csv('out.csv')

In [ ]:
def is_cont(col):
    if type(col) != np.int64:
        if type(col) == np.float64:
            return True
        if len(col) > 0 and type(col.iloc[0]) == np.float64:
            return True
    counts = col.value_counts()
    if (len(counts[counts < .1 * len(counts)])) > 0:
        return True
    return False

In [ ]:
def min_max_finder(data):
    max_col = dict()
    min_col = dict()
    for x in data.columns:
        max_col[x] = max(data[x])
        min_col[x] = min(data[x])
    return max_col, min_col

In [ ]:
def normalize(df, max_cols, min_cols, conts):
    def normal_func(x, col):
        if max_cols[col] - min_cols[col] == 0:
            return 0
        return 2 * (x - min_cols[col]) / (max_cols[col] - min_cols[col]) - 1
    for col in df.columns:
        if conts[col]:
            df[col] = df[col].apply(
                lambda x: normal_func(x, col))

In [ ]:
def convert_to_one_hot(col):
    encoder = OneHotEncoder(handle_unknown='ignore')
    col = col.reshape(-1, 1)
    encoder.fit(col)
    new_col = encoder.transform(col).toarray()
    return new_col

In [ ]:
def extract_dataframe(filename, delim=None, names=None):
    df = pd.read_csv(filename, delimiter=delim, names=names)
    class_name = [x for x in df.columns if 'class' in x.lower()][0]
    df.rename(columns={class_name: 'class'}, inplace=True)
    num_vals = dict()
    for x in df:
        num_vals[x] = np.array(sorted(df[x].unique()))

    return df, num_vals



In [ ]:
def make_X_y_split(df):
    X = df.loc[:, df.columns != 'class']
    y = df['class']
    X = np.array(X)
    y = np.array([y])
    y = convert_to_one_hot(y)
    return X, y

In [ ]:
def create_thetas(layers: list):
    thetas = dict()
    for i, v in enumerate(layers[:-1]):
        val = np.random.uniform(-1, 1, size=(layers[i+1], v+1))
        thetas[i] = val
    return thetas


In [ ]:
def add_bias(X):
    o = np.ones((np.shape(X)[0], 1), dtype=int)
    return np.column_stack((o, X))

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def z(weights: np.array, inputs: np.array):
    return np.matmul(weights, inputs)

In [ ]:
def run_forward_prop(X, theta):
    a = []
    a.append(np.asarray(add_bias(X)))
    for l in range(len(theta) - 1):
        # print(a[l])
        zl = np.matmul(a[l], theta[l].transpose())
        zl = sigmoid(zl)
        a.append(add_bias(zl))
    zl = np.matmul(a[len(theta)-1], theta[len(theta)-1].transpose())
    a.append(sigmoid(zl))
    return a

In [ ]:
def J(outputs, y, reg=0, lamb=0.0):
    tempRes1 = []
    tempRes2 = []
    with np.errstate(divide="ignore"):
        tempRes1 = np.log(outputs)
        tempRes2 = np.log(1-outputs)
    tempRes1[np.isneginf(tempRes1)] = 0
    tempRes2[np.isneginf(tempRes2)] = 0
    jx = np.multiply(-1 * y, tempRes1)
    jy = np.multiply(1 - y, tempRes2)
    j = jx - jy
    j = np.sum(j) / len(y)
    return np.mean(j) + (reg if lamb != 0 else 0)

In [ ]:
def run_back_prop(a, y, thetas, lamb):
    delta = dict()
    delta[len(thetas)] = np.subtract(a[-1], y)
    for k in range(len(thetas)-1, -1, -1):
        delta[k] = np.matmul(delta[k+1], thetas[k])
        delta[k] = np.multiply(delta[k], a[k])
        delta[k] = np.multiply(delta[k], 1 - a[k])
        delta[k] = np.delete(delta[k], (0), axis=1)
    D = dict()
    for k in range(len(thetas) - 1, -1, -1):
        if k not in D:
            D[k] = 0
        D[k] += np.matmul(delta[k+1].transpose(), a[k])
    for k in range(len(thetas)-1, -1, -1):
        if lamb != 0.0:
            Pk = lamb * thetas[k]
            Pk[:, 0] = np.array([0] * len(Pk[:, 0]))
            D[k] = (D[k] + Pk)
        D[k] /= len(y)
    return D

In [ ]:
def calc_regularization(thetas, y, lamb=0):
    S = 0
    if lamb != 0:
        for a in thetas.values():
            for b in a:
                S += sum(b[1:]**2)
        S = lamb * S / 2 / len(y)
    return S

In [ ]:
def gradient_descent(X, y, thetas, stop_crit, alpha, lamb):
    a = run_forward_prop(X, thetas)
    cur_cost = 100000
    reg = calc_regularization(thetas, y, lamb)
    new_c = J(a[-1], y, reg, lamb)
    num_iter = 0
    while (cur_cost - new_c > stop_crit and num_iter < 200000):
        cur_cost = new_c
        gradients = run_back_prop(a, y, thetas, lamb)
        for k in range(len(thetas)-1, -1, -1):
            thetas[k] -= alpha * gradients[k]
        num_iter += 1
        a = run_forward_prop(X, thetas)
        new_c = J(a[-1], y, reg, lamb)
        if num_iter % 10000 == 0:
            print(f"After {num_iter} iterations: {cur_cost - new_c}")
    print("Final Diff Cost:", cur_cost - new_c)
    print("Final Cost", new_c)
    print("Number of Iterations:", num_iter)
    return thetas

In [ ]:
def train_test_split(all_indices: dict, foldNum: int):
    train = []
    test = []
    for c in all_indices:
        test.extend(all_indices[c][foldNum])
        t = np.concatenate(
            all_indices[c][0:foldNum]+all_indices[c][foldNum+1:]).ravel()
        train.extend(t)
    return train, test

In [ ]:
def make_subsets(df, classes, num_splits: int):
    all = dict()
    for c in classes:
        all[c] = df.index[df['class'] == c].tolist()
        random.shuffle(all[c])
        all[c] = np.array_split(all[c], num_splits)
    return all

def convert_to_class(output):
    output = np.array(
        list((map(lambda x: np.where(x - max(x) >= 0, 1, 0), output))))
    return output

In [ ]:
def calc_accuracy(actual, predicted):
    # key = 'actual,predicted', value = frequency
    outputs = Counter({'0,0': 0, '0,1': 0, '1:0': 0, '1,1': 0})
    total = 0
    for i in range(len(actual)):
        trueIndex = int(np.where(actual[i] == 1)[0])
        predIndex = int(np.where(predicted[i] == 1)[0])
        if trueIndex == predIndex:
            total += 1
        outputs += Counter([f"{trueIndex},{predIndex}"])
    return outputs, total/len(actual)

def F1_score(precision, recall):
    return 2 * precision * recall / (precision + recall)

def precision(all, num_vals):
    fin_len = len(num_vals) - 1 if len(num_vals) == 2 else len(num_vals)
    num_vals = num_vals[:-1] if len(num_vals) == 2 else num_vals
    num_vals = num_vals - min(num_vals) if min(num_vals) > 0 else num_vals
    precisions = 0
    for i in num_vals:
        TP = all[f"{i},{i}"]
        FP = sum([val for x, val in all.items()
                  if f",{i}" in x and f"{i},{i}" not in x])
        precisions += TP/(TP + FP) if TP+FP != 0 else 0
    return precisions / fin_len

def recall(all, num_vals):
    fin_len = len(num_vals) - 1 if len(num_vals) == 2 else len(num_vals)
    num_vals = num_vals[:-1] if len(num_vals) == 2 else num_vals
    num_vals = num_vals - min(num_vals) if min(num_vals) > 0 else num_vals
    recalls = 0
    for i in num_vals:
        TP = all[f"{i},{i}"]
        FN = sum([val for x, val in all.items()
                  if f"{i}," in x and f"{i},{i}" not in x])
        recalls += TP/(TP + FN) if TP + FN != 0 else 0
    return recalls / fin_len

In [ ]:
def plots(x_axis, y_axis, y_lab, x_lab):
    plt.figure(figsize=(10, 6))
    plt.scatter(x_axis, y_axis, label=y_lab)
    plt.errorbar(x_axis, y_axis)
    plt.xlabel(x_lab)
    plt.ylabel(y_lab)
    plt.title(f"{y_lab} vs {x_lab}")
    plt.legend()
    plt.autoscale()
    plt.show()

In [ ]:
def run_cross_valid(df, num_vals, alpha, net, conts, lamb=0, k=10, find_cost=0):
    num = np.ceil(len(df['class']) / k)
    all_indices = make_subsets(df, num_vals['class'], k)
    print(f"Number of instances in test set: {num}")
    test_accuracies = []
    all_fScore = []
    layers = [len(df.columns) - 1]
    layers.extend(net)
    layers.append(len(num_vals['class']))
    thetas = create_thetas(layers)
    # print(all_indices)
    # return
    for i in range(k):
        print(
            f"-----Iteration {i+1}  Net {net} Lambda {lamb} -----------------")
        train, test = train_test_split(all_indices, i)
        # print("TRAIN", train)
        # print("TEST", test)
        train = df.iloc[train]
        test = df.iloc[test]
        if (len(test) == 0):
            break
        # test.reset_index(drop=True, inplace=True)

        max_cols, min_cols = min_max_finder(train)
        normalize(train, max_cols, min_cols, conts)
        normalize(test, max_cols, min_cols, conts)
        trainX, trainY = make_X_y_split(train)
        testX, testY = make_X_y_split(test)

        thetas = gradient_descent(trainX, trainY, thetas, SC, alpha, lamb)
        results = run_forward_prop(testX, thetas)[-1]
        results = convert_to_class(results)
        if find_cost == 0:
            op, ac = calc_accuracy(testY, results)

            print("Conf Matrix:", op)
            print("Accuracy:", ac)

            f1 = F1_score(precision(op, num_vals['class']), recall(
                op, num_vals['class']))
            print("F1-Score:", f1)
            all_fScore.append(f1)
            test_accuracies.append(ac)
        else:
            reg = calc_regularization(thetas, testY, lamb)
            t_cost = J(results, testY, reg, lamb=lamb)
            # print("TCOST TCOST---------------------------------", t_cost)
            test_accuracies.append(t_cost)
        # print('-------------------------', test_accuracies)

    return test_accuracies, all_fScore

In [ ]:
df, num_vals = extract_dataframe(
        'out.csv')
conts = dict()
for x in df:
    conts[x] = is_cont(df[x])

In [52]:

SC = 1e-4
with ThreadPoolExecutor(8) as executor:
    

    for l in [.1, .01]:
        fin_accuracy = []
        fin_fscore = []

        for num_hid in [1, 3]:
            for num_neur in [4, 8, 16, 25, 32, 64]:
                accs, fscores = run_cross_valid(
                    df, num_vals, alpha=.1, net=[num_neur] * num_hid, conts=conts, lamb=l)
                # print(f"Accuracies: {accs},\nMean: {np.mean(accs)}")
                fin_accuracy.append(np.round(np.mean(accs), decimals=4))
                # print(f"F1-Scores: {fscores},\nMean: {np.mean(fscores)}")
                fin_fscore.append(np.round(np.mean(fscores), decimals=4))
                # break
            # break
        # print()
        # results_table(fin_accuracy)
        # print()
        # results_table(fin_fscore)
        # print()
        print(fin_accuracy)
        print(fin_fscore)
    


Number of instances in test set: 61.0
-----Iteration 1  Net [4] Lambda 0.1 -----------------


C:\Users\Tanishk\AppData\Local\Temp/ipykernel_18588/869318862.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(


Final Diff Cost: 9.465531524810444e-05
Final Cost 1.399772653192838
Number of Iterations: 30
Conf Matrix: Counter({'0,0': 33, '1,0': 27, '0,1': 1, '1,1': 1})
Accuracy: 0.5483870967741935
F1-Score: 0.7021276595744681
-----Iteration 2  Net [4] Lambda 0.1 -----------------


C:\Users\Tanishk\AppData\Local\Temp/ipykernel_18588/869318862.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(


Final Diff Cost: 9.268249229421599e-05
Final Cost 1.4021509620590045
Number of Iterations: 8
Conf Matrix: Counter({'0,0': 33, '1,0': 26, '0,1': 1, '1,1': 1})
Accuracy: 0.5573770491803278
F1-Score: 0.7096774193548386
-----Iteration 3  Net [4] Lambda 0.1 -----------------


...

Final Diff Cost: -0.0483619820980814
Final Cost 1.5075599072675436
Number of Iterations: 2
Conf Matrix: Counter({'0,0': 31, '1,0': 24, '1,1': 3, '0,1': 2})
Accuracy: 0.5666666666666667
F1-Score: 0.7045454545454546
-----Iteration 9  Net [64, 64, 64] Lambda 0.01 -----------------


C:\Users\Tanishk\AppData\Local\Temp/ipykernel_18588/869318862.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(


Final Diff Cost: -0.012836363962118513
Final Cost 1.4380050202545327
Number of Iterations: 2
Conf Matrix: Counter({'0,0': 31, '1,0': 25, '0,1': 2, '1,1': 2})
Accuracy: 0.55
F1-Score: 0.6966292134831461
-----Iteration 10  Net [64, 64, 64] Lambda 0.01 -----------------


C:\Users\Tanishk\AppData\Local\Temp/ipykernel_18588/869318862.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].apply(


Final Diff Cost: -0.010600122415387325
Final Cost 1.4371300014090387
Number of Iterations: 2
Conf Matrix: Counter({'0,0': 33, '1,0': 22, '1,1': 5})
Accuracy: 0.6333333333333333
F1-Score: 0.7499999999999999
[0.551, 0.6299, 0.6578, 0.6924, 0.6894, 0.5965, 0.5543, 0.5543, 0.5526, 0.5527, 0.5826, 0.5954]
[0.708, 0.6849, 0.7036, 0.7177, 0.7217, 0.6057, 0.7132, 0.7132, 0.7112, 0.7112, 0.727, 0.7087]
